**Uploading the ZIP file from our local system**

In [1]:
from google.colab import files
uploaded = files.upload()


Saving Skin.zip to Skin.zip


**Unzip The DATASET**

In [2]:
import zipfile
import os

zip_path = "/content/Skin.zip"
extract_dir = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ Unzipped to:", extract_dir)
os.listdir(extract_dir)


✅ Unzipped to: /content/dataset


['Acne and Rosacea Photos',
 'Hair Loss Photos Alopecia and other Hair Diseases',
 'Melanoma Skin Cancer Nevi and Moles',
 'Light Diseases and Disorders of Pigmentation',
 'Herpes HPV and other STDs Photos',
 'Nail Fungus and other Nail Disease']

**Checking the folder structure**

In [3]:
import os

# Check the first-level folders inside the extracted directory
for item in os.listdir(extract_dir):
    item_path = os.path.join(extract_dir, item)
    print(f"{item}/ - {'DIR' if os.path.isdir(item_path) else 'FILE'}")


Acne and Rosacea Photos/ - DIR
Hair Loss Photos Alopecia and other Hair Diseases/ - DIR
Melanoma Skin Cancer Nevi and Moles/ - DIR
Light Diseases and Disorders of Pigmentation/ - DIR
Herpes HPV and other STDs Photos/ - DIR
Nail Fungus and other Nail Disease/ - DIR


**Splitting into TRAIN & VAL**

In [8]:
import shutil
from sklearn.model_selection import train_test_split

output_dir = "/content/split_dataset"  # Final dataset with train/val folders
val_ratio = 0.2  # 20% for validation

# Create the directory structure
for split in ['train', 'val']:
    for class_name in os.listdir(base_dir):
        class_path = os.path.join(base_dir, class_name)
        if os.path.isdir(class_path):
            os.makedirs(os.path.join(output_dir, split, class_name), exist_ok=True)

# Split and copy images
for class_name in os.listdir(base_dir):
    class_path = os.path.join(base_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    train_imgs, val_imgs = train_test_split(images, test_size=val_ratio, random_state=42)

    for img in train_imgs:
        shutil.copy(os.path.join(class_path, img), os.path.join(output_dir, 'train', class_name, img))

    for img in val_imgs:
        shutil.copy(os.path.join(class_path, img), os.path.join(output_dir, 'val', class_name, img))

print("✅ Dataset split complete. Ready for training at:", output_dir)


✅ Dataset split complete. Ready for training at: /content/split_dataset


**Training the CLASSIFICATION MODEL**

In [10]:
!pip install ultralytics # Installing ULTRALYTICS in Colab
from ultralytics import YOLO

model = YOLO('yolov8n-cls.pt')
model.train(data=output_dir, epochs=25, imgsz=224, batch=32)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

100%|██████████| 5.31M/5.31M [00:00<00:00, 90.1MB/s]


Ultralytics 8.3.143 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (AMD EPYC 7B12)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/split_dataset, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, prof

train: Scanning /content/split_dataset/train... 792 images, 0 corrupt: 100%|██████████| 792/792 [00:01<00:00, 416.14it/s]

train: New cache created: /content/split_dataset/train.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 46.4±28.2 MB/s, size: 159.7 KB)


...

                   all      0.797      0.985



25 epochs completed in 0.231 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 3.0MB
Optimizer stripped from runs/classify/train/weights/best.pt, 3.0MB

Validating runs/classify/train/weights/best.pt...
Ultralytics 8.3.143 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (AMD EPYC 7B12)
YOLOv8n-cls summary (fused): 30 layers, 1,442,566 parameters, 0 gradients, 3.3 GFLOPs
train: /content/split_dataset/train... found 792 images in 6 classes ✅ 
val: /content/split_dataset/val... found 202 images in 6 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 4/4 [00:03<00:00,  1.30it/s]


                   all      0.832       0.99
Speed: 0.0ms preprocess, 8.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7efb57249c10>
curves: []
curves_results: []
fitness: 0.9108910858631134
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8316831588745117, 'metrics/accuracy_top5': 0.9900990128517151, 'fitness': 0.9108910858631134}
save_dir: PosixPath('runs/classify/train')
speed: {'preprocess': 0.000396089109638677, 'inference': 7.956953381188547, 'loss': 2.4455445809237947e-05, 'postprocess': 5.762376193809243e-05}
task: 'classify'
top1: 0.8316831588745117
top5: 0.9900990128517151

In [12]:
# 5. Evaluate on validation set
metrics = model.val()
print(metrics)


Ultralytics 8.3.143 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (AMD EPYC 7B12)
YOLOv8n-cls summary (fused): 30 layers, 1,442,566 parameters, 0 gradients, 3.3 GFLOPs
train: /content/split_dataset/train... found 792 images in 6 classes ✅ 
val: /content/split_dataset/val... found 202 images in 6 classes ✅ 
test: None...
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1180.4±725.2 MB/s, size: 159.7 KB)


val: Scanning /content/split_dataset/val... 202 images, 0 corrupt: 100%|██████████| 202/202 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 7/7 [00:03<00:00,  1.98it/s]


                   all      0.832       0.99
Speed: 0.0ms preprocess, 8.8ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2
ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7efb66f16f90>
curves: []
curves_results: []
fitness: 0.9108910858631134
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8316831588745117, 'metrics/accuracy_top5': 0.9900990128517151, 'fitness': 0.9108910858631134}
save_dir: PosixPath('runs/classify/train2')
speed: {'preprocess': 0.0007307871287999246, 'inference': 8.818614252474239, 'loss': 4.504455535083036e-05, 'postprocess': 0.0001296485147622211}
task: 'classify'
top1: 0.8316831588745117
top5: 0.9900990128517151
